<h1>EDA to get buyer and seller valuation</h1>

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [57]:
# Load excel file into a DataFrame
df = pd.read_excel("../datasets/resalepricesadjusted.xlsx")

df.head()
df['Neighborhood score'] = df['Neighborhood score'].astype(float)
df['flat_type'] = df['flat_type'].astype(float)
df['remaining_lease'] = df['remaining_lease'].astype(float)
df['resale_price'] = df['resale_price'].astype(float)
df['mean price'] = df['mean price'].astype(float)
df['ln_resale_price'] = df['ln_resale_price'].astype(float)
df['ln_mean_price'] = df['ln_mean_price'].astype(float)

In [58]:
# Group by 'neighborhood score' and calculate the average resale price
neighborhood_avg_resale_price = df.groupby('month')['resale_price'].transform('mean')

# Create a new column 'average_resale_price' to store the average resale price for each neighborhood score
df['average_resale_price'] = neighborhood_avg_resale_price

In [59]:
# delete all rows where month does not include 2019
df = df[df['month'].str.contains("2019")]
print(df.head())

         month        town  Neighborhood score  flat_type  \
42051  2019-01  ANG MO KIO                 4.0        3.0   
42052  2019-01  ANG MO KIO                 4.0        3.0   
42053  2019-01  ANG MO KIO                 4.0        3.0   
42054  2019-01  ANG MO KIO                 4.0        3.0   
42055  2019-01  ANG MO KIO                 4.0        3.0   

       lease_commence_date  remaining_lease  resale_price  mean price  \
42051                 1978             58.0      230000.0   386977.03   
42052                 1986             66.0      235000.0   386977.03   
42053                 1979             59.0      238000.0   386977.03   
42054                 1986             66.0      240000.0   386977.03   
42055                 1980             60.0      240000.0   386977.03   

       ln_resale_price  ln_mean_price  average_resale_price  
42051            12.35      12.866121         429810.536129  
42052            12.37      12.866121         429810.536129  
42053   

In [60]:
# add monthly income column 
# 2 room is 2521	
# 3 room is 5868
# 4 room is 8827
# 5 room is 12244

income_mapping = {
    1: 1500,
    2: 2521,
    3: 5868,
    4: 8827,
    5: 12244,
    6: 12244,
}

df['monthly_income'] = df['flat_type'].map(income_mapping)

df.head()

,month,town,Neighborhood score,flat_type,lease_commence_date,remaining_lease,resale_price,mean price,ln_resale_price,ln_mean_price,average_resale_price,monthly_income
42051,2019-01,ANG MO KIO,4.0,3.0,1978,58.0,230000.0,386977.03,12.35,12.866121,429810.536129,5868
42052,2019-01,ANG MO KIO,4.0,3.0,1986,66.0,235000.0,386977.03,12.37,12.866121,429810.536129,5868
42053,2019-01,ANG MO KIO,4.0,3.0,1979,59.0,238000.0,386977.03,12.38,12.866121,429810.536129,5868
42054,2019-01,ANG MO KIO,4.0,3.0,1986,66.0,240000.0,386977.03,12.39,12.866121,429810.536129,5868
42055,2019-01,ANG MO KIO,4.0,3.0,1980,60.0,240000.0,386977.03,12.39,12.866121,429810.536129,5868


In [61]:
# Assuming df is your DataFrame
# Selecting only the specified columns from the dataframe
X_columns = ['ln_mean_price', 'flat_type', 'remaining_lease', 'monthly_income']
X = df[X_columns]

# Find rows with NaN values in the specified columns
rows_with_nan = X[X.isna().any(axis=1)]

# Display rows with NaN values
print(rows_with_nan)

Empty DataFrame
Columns: [ln_mean_price, flat_type, remaining_lease, monthly_income]
Index: []


In [62]:
X_columns = ['ln_mean_price','flat_type','remaining_lease','monthly_income']
# Selecting only the specified columns from the dataframe
X = df[X_columns]

# Splitting the dataset into features and target variable
y = df['ln_resale_price'] # Target variable

# Splitting the dataset into the training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [63]:
# Making predictions
y_pred = model.predict(X_test)

In [64]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

# Coefficients of the model
coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
coefficients_str = coefficients.to_string(float_format='{:f}'.format)
print("\nCoefficients:")
print(coefficients_str)

# Intercept of the model
print("\nIntercept:", model.intercept_)

Mean Squared Error: 0.030422650056284535
R-squared: 0.7333829239503342

Coefficients:
                 Coefficient
ln_mean_price       1.007892
flat_type           0.206993
remaining_lease     0.006349
monthly_income      0.000013

Intercept: -1.5865776276451484


<h1>Formula</h1>

$$
\ln(\text{offer price}) = 1.01\ln(\text{mean price}) +
 (0.207\space\text{number of rooms}) + (0.00635\space
  \text{remaining lease}) + (0.000013 \space \text{Monthly Income}) - 1.59
$$

<h1>Seller Valuation</h1>

In [65]:
X_columns = ['ln_mean_price', 'remaining_lease']
# Selecting only the specified columns from the dataframe
X = df[X_columns]

# Splitting the dataset into features and target variable
y = df['ln_resale_price'] # Target variable

# Splitting the dataset into the training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [66]:
y_pred = model.predict(X_test)

In [67]:
# Model evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

# Coefficients of the model
coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
print("\nCoefficients:")
print(coefficients)

# Intercept of the model
print("\nIntercept:", model.intercept_)

Mean Squared Error: 0.07728793734953905
R-squared: 0.3226663741679159

Coefficients:
                 Coefficient
ln_mean_price       0.976328
remaining_lease     0.009917

Intercept: -0.4740118524310777


<h1>Formula</h1>

$$
\ln (\text{resale price} )= (0.976 \space \ln(\text{mean price}) + 0.00992 \space \text{remaining lease}) - 0.474
$$